In [ ]:
# https://huggingface.co/docs/transformers/main_classes/pipelines
# https://huggingface.co/docs/diffusers/en/api/pipelines/overview
# https://huggingface.co/models

!pip install -q -U transformers datasets diffusers soundfile

import torch
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio

device = "cuda" if torch.cuda.is_available() else "cpu"
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# sentiment analysis
classifier = pipeline("sentiment-analysis", device=device)
result = classifier("A diamond in the rough in the form of a star who appeared like a comet—Idol VTuber Hoshimachi Suisei!")
print("Sentiment Analysis:", result)

In [ ]:
# named entity recognition
ner = pipeline("ner", grouped_entities=True, device=device)
result = ner("Hoshimachi Suisei's YouTube channel was created on 18 March 2018")
print("Named Entity Recognition:", result)

In [ ]:
# question answering with context
question_answerer = pipeline("question-answering", device=device)
result = question_answerer(question="When did Hoshimachi Suisei create her YouTube channel?", context="Hoshimachi Suisei's YouTube channel was created on 18 March 2018")
print("Question Answering:", result)

In [ ]:
# text summarization
summarizer = pipeline("summarization", device=device)
text = """
Hoshimachi Suisei (星街すいせい) is a Japanese singer and virtual YouTuber. She began posting videos as an independent creator in March 2018.
In May 2019, she became affiliated with Hololive Production through their newly created music label, INoNaKa Music, before joining the agency's main branch later the same year.
Her YouTube activity consists primarily of live streaming herself singing karaoke, playing video games, talking to her fans, or collaborating with other talents.
She is particularly well known among fans for her skill at Tetris and her singing ability. As of March 2024, she has 2.11 million subscribers and 731 million views on YouTube.
"""
summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
print("Summary:", summary[0]['summary_text'])

In [ ]:
# translation
# this prompt breaks the translation
translator = pipeline("translation", model="staka/fugumt-en-ja", device=device)
result = translator("Mark Zuckerberg has been aggressively recruiting top talent from OpenAI as part of his efforts to establish Meta as a leader in artificial intelligence.")
print("Translate English to Japanese:", result[0]['translation_text'])

In [ ]:
# zero shot classification
classifier = pipeline("zero-shot-classification", device=device)
result = classifier("Shuchao Bi: Co-creator of GPT-4o voice mode and o4-mini, previously leading multimodal post-training at OpenAI, now part of Meta's team.",
                    candidate_labels=["technology", "sports", "politics"])
print("Zero-Shot Classification:", result)

In [ ]:
# text gen
# this prompt breaks the text generator
generator = pipeline("text-generation", device=device)
result = generator("The key things to know about HuggingFace pipelines are:")
print(result[0]['generated_text'])

In [ ]:
# image gen
# tokyo dome doesn't look like that
image_gen = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
).to(device)
prompt = "Show me a picture of Tokyo Dome"
image = image_gen(prompt=prompt).images[0]
image

In [ ]:
# free gpu memory
del image_gen
torch.cuda.empty_cache()

In [ ]:
# audio generation / text to speech
# this prompt breaks the audio generator
synthesiser = pipeline("text-to-speech", model="microsoft/speecht5_tts", device=device)
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
speech = synthesiser("""
Zuckerberg has personally reached out to desired candidates, aiming to create a transfusion from the country’s top AI labs.
The Meta CEO has also promised to pivot the company into an "AI-first" organization, leading to the creation of Meta Superintelligence Labs (MSL),
which includes all of Meta's foundations, product, and FAIR teams, as well as a new lab focused on developing the next generation of models.
""", forward_params={"speaker_embeddings": speaker_embedding})
sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
display(Audio("speech.wav"))